# Импорты

In [2]:
import os
import pandas as pd
import re

# Предобработка данных

Посмотрим на текущие данные. Для примера возьмём 1 таблицу каталога и 1 таблицу отзыва, чтобы понять из чего состоят данные

In [3]:
df_catalog = pd.read_csv('./../data/clean/catalogs/obuvmuzhskayakedy_i_krossovki.csv')
df_review = pd.read_csv('./../data/clean/reviews/obuvmuzhskayakedy_i_krossovki/6153738.csv')

In [5]:
df_catalog.head()

,product_article,product_name,product_rating,count_reviews
0,259646173,Кроссовки мужские зимние с мехом высокие,"4,6",100
1,252655283,кроссовки зимние,"4,7",100
2,250220847,Кроссовки утепленные с мехом,"4,7",100
3,265989233,Кеды vans knu mid Зима,"4,6",100
4,258468744,Кроссовки зимние высокие,"4,3",100


In [6]:
df_review.head()

,review_rating,review_text,review_text_plus,review_text_minus
0,5,Достоинства:Качество хорошее.,NaN,NaN
1,5,"Достоинства:размер подошел, на ноге сидят отли...",NaN,NaN
2,5,Достоинства:Хороший кроссовки\nНедостатки:Их нет,NaN,NaN
3,5,"Достоинства:Мделано добротно, как т предшестве...",NaN,NaN
4,5,Достоинства:Товар хороший\nНедостатки:Таблица ...,NaN,NaN


Поскольку результатом моей работы должен быть `Word Cloud`, то обработка должна включать в себя разделение review_text на 3 столбика:
- Основной текст
- Достоинства
- Недостатки

Пока выполним преобразования для одного каталога, так как он всего 1 на данный момент. В будущем автоматизируем для всех каталогов

In [18]:
path_reviews = './../data/clean/reviews/obuvmuzhskayakedy_i_krossovki'
files_reviews = os.listdir(path_reviews)

# Функция для парсинга текста
def parse_review(review):
    # Убедимся, что входные данные являются строкой
    if not isinstance(review, str):
        review = ""  # Преобразуем в пустую строку, если это не строка

    # Регулярные выражения для поиска каждого компонента
    plus_pattern = r"Достоинства:\s*(.*?)(?=\n|$)"
    minus_pattern = r"Недостатки:\s*(.*?)(?=\n|$)"
    comment_pattern = r"Комментарий:\s*(.*?)(?=\n|$)"
    
    # Поиск компонентов с использованием регулярных выражений
    review_plus = re.search(plus_pattern, review)
    review_minus = re.search(minus_pattern, review)
    review_comment = re.search(comment_pattern, review)

    # Если не найдены стандартные комментарии, всё, что не в plus и minus, считаем за комментарий
    if not review_comment:
        review_comment = re.search(r"^(?!Достоинства:|Недостатки:)(.*)", review.strip())
    
    # Возвращаем найденные компоненты, если они есть, иначе "Нет"
    return {
        'review_text_plus': review_plus.group(1) if review_plus else 'Нет',
        'review_text_minus': review_minus.group(1) if review_minus else 'Нет',
        'review_text_comment': review_comment.group(1) if review_comment else 'Нет'
    }

for f_review in files_reviews:
    df_reviews = pd.read_csv(f'{path_reviews}/{f_review}')
    df_reviews = df_reviews.drop(['review_text_plus', 'review_text_minus'], axis=1)
    parsed_reviews = df_reviews['review_text'].apply(parse_review).apply(pd.Series)
    df_reviews = pd.concat([df_reviews, parsed_reviews], axis=1)
    df_reviews.to_csv(f'./../data/processed/reviews/obuvmuzhskayakedy_i_krossovki/{f_review}', index=False)

Объединим все отзывы в 1 таблицу

In [8]:
path_reviews = './../data/processed/reviews/obuvmuzhskayakedy_i_krossovki'
files_reviews = os.listdir(path_reviews)
list_reviews_df = []

for f_review in files_reviews:
    df_review = pd.read_csv(f'{path_reviews}/{f_review}')
    list_reviews_df.append(df_review)

df_all_reviews = pd.concat(list_reviews_df, ignore_index=True)
df_all_reviews = df_all_reviews.drop(['review_text.1'], axis=1)
df_all_reviews.to_csv(f'./../data/processed/reviews/obuvmuzhskayakedy_i_krossovki/all.csv', index=False)